In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2
import os
import matplotlib.image as mpimg

In [19]:
#caracters de sortida:
sortides = [1,2,3,4,5]
sortides

[1, 2, 3, 4, 5]

In [22]:
#Mostrar algunas imagenes con pyplot (las 15 primeras)
plt.figure(figsize=(15,9))
imagenes = np.array([])
carpeta = 'data/data'
for i in range(5):
    imagenes = np.append(os.listdir(f"../dataset/sign_{i}/"))

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '../dataset/sign_0/'

<Figure size 1500x900 with 0 Axes>

In [ ]:
import tensorflow as tf
from tensorflow.python import metrics
#Modelo con funcion de costo Error Cuadratico Medio (MSE) One Hot Encoded (OHE)
mseModelOHE = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[64,64,1]), #neuronas de entrada
    tf.keras.layers.Dense(50, activation=tf.nn.relu), #1a capa oculta activacion relu
    tf.keras.layers.Dense(15, activation=tf.nn.softmax), #capa de salida 15 salidas posibles
])
#Compilar el modelo
mseModelOHE.compile(    
    optimizer = 'adam',
    loss='mean_squared_error',
    metrics = ['accuracy']
)
mseModelOHE.summary()

#Modelo con funcion de costo Porcentage de error absoluto medio (MAPE) One Hot Encoded (OHE)

mapeModelOHE = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[64,64,1]), #neuronas de entrada
    tf.keras.layers.Dense(50, activation=tf.nn.relu), #1a capa oculta activacion relu
    tf.keras.layers.Dense(15, activation=tf.nn.softmax), #capa de salida 15 salidas posibles
])
#Compilar el modelo
mapeModelOHE.compile(    
    optimizer = 'adam',
    loss=tf.keras.losses.MeanAbsolutePercentageError(),
    metrics = ['accuracy']
)
mapeModelOHE.summary()

#Modelo con funcion de costo categorical_crossentropy One Hot Encoded (OHE)

cceModelOHE = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[64,64,1]), #neuronas de entrada
    tf.keras.layers.Dense(50, activation=tf.nn.relu), #1a capa oculta activacion relu
    tf.keras.layers.Dense(15, activation=tf.nn.softmax), #capa de salida 15 salidas posibles
])
#Compilar el modelo
cceModelOHE.compile(    
    optimizer = 'adam',
    loss='categorical_crossentropy',
    metrics = ['accuracy']
)
cceModelOHE.summary()

#Modelo con funcion de costo Porcentage de error absoluto medio (MAPE) One Hot Encoded (OHE)

maeModelOHE = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[64,64,1]), #neuronas de entrada
    tf.keras.layers.Dense(50, activation=tf.nn.relu), #1a capa oculta activacion relu
    tf.keras.layers.Dense(15, activation=tf.nn.softmax), #capa de salida 15 salidas posibles
])
#Compilar el modelo
maeModelOHE.compile(    
    optimizer = 'adam',
    loss=tf.keras.losses.MeanAbsoluteError(),
    metrics = ['accuracy']
)
maeModelOHE.summary()